# Python-Beispiele für RIS-API

This notebook contains some examples to show how to work with the RIS API.
Dieses Notebook beinhaltet einige Python-Beispiele, die zeigen, wie man mit der RIS-API arbeiten kann.

Inhaltsverzeichnis:

1. [Mitglieder einer Partei](#Mitglieder-einer-Partei)
1. [Gemeinderats-Geschäfte](#Gemeinderats-Geschäfte)

In [1]:
import requests
from pprint import pprint
import pandas as pd
import random

## Mitglieder einer Partei

Um alle Mitglieder einer Partei zu bekommen, müssen folgende Abfragen gemacht werden:
    
    1. ID der Partei holen
    1. Suche nach Mitgliedern der Partei (mit dieser ID)
    1. *Optional:* Detail-Informationen eines Mitglieds holen
    
    

### Alle Parameter holen

In [11]:
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/parameter', headers=headers)
params = r.json()

SSLError: HTTPSConnectionPool(host='www.gemeinderat-zuerich.ch', port=443): Max retries exceeded with url: /api/Mitglieder/parameter (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [4]:
# get id of random party
parties = params['Parteien']
party = random.choice(parties)
party

{'Name': 'EVP', 'Id': '187b7ad1-2095-43cc-b257-e9834a54a961'}

### Mitglieder der Partei abfragen

In [6]:
query = {'parteiId': party['Id']}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/suchen', params=query, headers=headers, verify=False)
members = r.json()
members

H:\Applikationen\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'Id': 'f15f6a61-bca6-4d3e-8109-c9e96950b433',
  'Name': 'Anderegg',
  'Vorname': 'Peter',
  'Titel': None,
  'Partei': 'EVP',
  'Wahlkreis': '11',
  'WahlkreisOrderBy': 107},
 {'Id': '63c8f7d9-abd1-4aae-8106-bfde63980439',
  'Name': 'Danner',
  'Vorname': 'Ernst',
  'Titel': '',
  'Partei': 'EVP',
  'Wahlkreis': '11',
  'WahlkreisOrderBy': 107},
 {'Id': '8ef1bf9d-63aa-4999-9ac6-0b3ee16c838f',
  'Name': 'Föhn',
  'Vorname': 'Roger',
  'Titel': None,
  'Partei': 'EVP',
  'Wahlkreis': '12',
  'WahlkreisOrderBy': 108},
 {'Id': 'dad38e55-68cc-445b-9648-de33fc11cd65',
  'Name': 'Rabelbauer',
  'Vorname': 'Claudia',
  'Titel': None,
  'Partei': 'EVP',
  'Wahlkreis': ' 9',
  'WahlkreisOrderBy': 105}]

### Details der Mitglieder abfragen

Mit dem `/Mitglieder/details` Endpunkt können Details eines Mitglieds abgefragt werden:

In [7]:
def get_member_details(member):
    query = {'mid': member['Id']}
    headers = {'Accept': 'application/json'}
    r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/details', params=query, headers=headers)
    details = r.json()
    # trim "Wahlkreis" and "Wohnkreis" since they have a padding
    details['Wahlkreis'] = details['Wahlkreis'].strip()
    if details['Wohnkreis']:
        details['Wohnkreis'] = details['Wohnkreis'].strip()
    # add original member to keep WahlkreisOrderBy column
    details.update(member)
    return details
    
member_details = [get_member_details(m) for m in members]
df = pd.DataFrame(member_details)
df

,Adressen,Anrede,Beruf,EmailGeschaeftlich,EmailPrivat,Fraktion,Geburtstag,GruppenMitgliedschaften,Id,Interessenverbindungen,...,NameInUrl,Partei,Sitznummer,TelefonGeschaeftlich,TelefonPrivat,Titel,Vorname,Wahlkreis,WahlkreisOrderBy,Wohnkreis
0,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Frau,Studentin,None,ezgi.akyol@gmx.ch,AL,1986-09-11T00:00:00,"[{'Von': '2014-05-07T00:00:00', 'Bis': None, '...",afb2e0bc-b941-4fa4-8936-9b4978fc1ee1,None,...,Ezgi%20Akyol,AL,None,None,None,None,Ezgi,4 und 5,102,4
1,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Herr,Leiter Kommunikation,walterangst@gmail.com,None,AL,1961-09-22T00:00:00,"[{'Von': '2002-04-10T00:00:00', 'Bis': None, '...",ee8204f4-9290-49ca-99ae-d18a22a0a513,"- Mieterinnen– und Mieterverband Zürich, Leite...",...,Walter%20Angst,AL,None,044 296 90 30,None,None,Walter,3,101,3
2,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Frau,Soziale Quartierentwicklung,quartiernetz@qnf.ch,,AL,1967-04-24T00:00:00,"[{'Von': '2018-05-16T00:00:00', 'Bis': None, '...",bbfb8ca0-91b1-448d-b1fc-c5e3bb35c158,"- Quartiernetz Friesenberg, Stellenleitung",...,Natalie%20Eberle,AL,None,043 537 80 17,,None,Natalie,3,101,9
3,"[{'Addressart': 'Postadresse', 'Strasse1': '',...",Herr,Arzt,david.garcia@al-zh.ch,,AL,1975-05-16T00:00:00,"[{'Von': '2017-01-19T00:00:00', 'Bis': None, '...",4a1be35e-3f69-48a4-85b1-3fee07b006ae,"- Volkshaus Zürich, Mitglied des Stiftungsrats",...,David%20Garcia%20Nu%c3%b1ez,AL,None,,,Dr.,David,4 und 5,102,1
4,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Herr,Stv. Bibliotheksdirektor,,andreas@kirstein.ch,AL,1963-05-08T00:00:00,"[{'Von': '2012-07-12T00:00:00', 'Bis': None, '...",b84bdf09-0197-4eac-989f-b6e49b297edd,"- Allg. Baugenossenschaft Zürich, Zürich, Vize...",...,Andreas%20Kirstein,AL,None,,,None,Andreas,11,107,11
5,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Frau,"Berufsschullehrerin, Leiterin Bildungsgang",,acaleitner@gmail.com,AL,1964-07-21T00:00:00,"[{'Von': '2013-05-29T00:00:00', 'Bis': None, '...",7f22b482-38d0-43ca-9599-a71f3edcbd73,- Stiftung Wohnungen für kinderreiche Familien...,...,Andrea%20Leitner%20Verhoeven,AL,None,None,,None,Andrea,10,106,10
6,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Herr,"Koch, Journalist",,paedemail@yahoo.com,AL,1965-06-07T00:00:00,"[{'Von': '2019-01-25T00:00:00', 'Bis': None, '...",9e46c2e0-626a-4d19-82e8-de1b2a3e216d,"- Quartierverein Wipkingen, Vorstand, Ressort ...",...,Patrik%20Maillard,AL,None,,,None,Patrik,6,103,10
7,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Frau,"Pädagogin, Imkerin",None,olivia@romanelli.ch,AL,1974-12-09T00:00:00,"[{'Von': '2019-02-11T00:00:00', 'Bis': None, '...",ad19c2cb-10d3-426a-a915-45763615ca0c,"- BienenSchweiz, Sektion ZH 101, Betriebsberat...",...,Olivia%20Romanelli,AL,None,None,,None,Olivia,1 und 2,100,2
8,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Frau,Studentin,None,christina.schiller@bluewin.ch,AL,1989-06-04T00:00:00,"[{'Von': '2014-05-07T00:00:00', 'Bis': None, '...",d2faaa00-3795-451c-9f72-1d17ebee399b,"VI Holding, Verwaltungsratsmitglied\r- Fondazi...",...,Christina%20Schiller,AL,None,None,None,None,Christina,9,105,9
9,"[{'Addressart': 'Postadresse', 'Strasse1': Non...",Herr,Filmwissenschaftler,micha.schiwow@prochaine.ch,mschiwow@gmx.ch,AL,1961-08-31T00:00:00,"[{'Von': '2016-06-28T00:00:00', 'Bis': None, '...",ddcd41f6-9873-4002-80d8-dfd87114c9a5,"- Verein Roadmovie Lausanne, Mitglied des Vors...",...,Mischa%20Schiwow,AL,None,044 488 44 25,044 262 60 23,None,Mischa,7 und 8,104,7


### Diagramm aller Mitglieder einer Partei nach Wahlkreis

In [8]:
count_by_wahlkreis = df[['Wahlkreis', 'WahlkreisOrderBy']].groupby(['Wahlkreis', 'WahlkreisOrderBy']).size().reset_index(name='Anzahl')
count_by_wahlkreis = count_by_wahlkreis.sort_values(by=['WahlkreisOrderBy'])
count_by_wahlkreis.plot.bar(x='Wahlkreis', y='Anzahl')

## Gemeinderats-Geschäfte